In [1]:
'''Benchmark #2'''

'''
add onto benchmark #1 by now introducing a few shape objects 
into the system

the canvas algorithm will go, and then the shape algorithm will go

test to see how much the evolution is affected by the order of 
shapes placed onto the canvas

shape objects can vary in size, location, and RGBA color
'''

import math
import random
import imgcompare
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw

In [2]:
class Hyperparameters:
    '''
    '''
    HYPERPARAMETER = 0

In [3]:
class Canvas:
    '''
    - handles only the background color of the painting
    '''
    def __init__(self) -> None:
        self.color = self.define_color()
    
    def __str__(self) -> str:
        return "Color: {}".format(self.color)
    
    def define_color(self) -> list:
        R = random.randint(0, 255)
        G = random.randint(0, 255)
        B = random.randint(0, 255)
        return [R, G, B]


if __name__ == "__main__":
    canvases = [Canvas() for _ in range(0, 10)]
    for canvas in canvases: print("{}".format(canvas))

Color: [130, 1, 88]
Color: [207, 123, 211]
Color: [195, 19, 123]
Color: [13, 78, 85]
Color: [71, 209, 220]
Color: [124, 53, 58]
Color: [177, 231, 25]
Color: [93, 54, 148]
Color: [207, 234, 81]
Color: [105, 17, 111]


In [4]:
class Shape:
    '''
    - circles
    '''
    def __init__(self, width: int, height: int, factor: float) -> None:
        self.width = width
        self.height = height
        self.factor = factor
        self.color = self.define_color()
        self.radius = int(0.5 * random.gammavariate(self.factor, 0.1 * self.get_hypotenuse()))
        self.center = [random.randint(0, self.width - 1), random.randint(0, self.height - 1)]
    
    def __str__(self) -> str:
        '''
        - reads out key specs fo the shape -> (color, center coordinate, radius)
        '''
        return "Color: {0}; Radius: {1}; Center: {2}".format(self.color, self.radius, self.center)
    
    def crossover(self):
        pass
    
    def define_color(self) -> list:
        R = random.randint(0, 255)
        G = random.randint(0, 255)
        B = random.randint(0, 255)
        A = random.randint(0, 255)
        return [R, G, B, A]
    
    def define_drawing_coordinates(self) -> list:
        '''
        - the drawing tool requires the shape's boundary coordinates in order to draw
        - the shape's center coordinate and radius are used to compute the boundary points
        '''
        x0 = self.center[0] - self.radius # 180 deg position
        x1 = self.center[0] + self.radius #   0 deg position
        y0 = self.center[1] - self.radius # 270 deg position
        y1 = self.center[1] + self.radius #  90 deg position
        return [x0, y0, x1, y1]
    
    def draw_circle(self, drawing_tool: object) -> None:
        '''
        - using the PIL library, this function draws an ellipse with symmetrical 
          coordinates, thus drawing a circle
        - the drawing tool is defined in the Painting object
        '''
        drawing_tool.ellipse(tuple(self.define_drawing_coordinates()), tuple(self.color))
    
    def get_hypotenuse(self) -> int:
        return int(math.sqrt((self.width) ** 2 + (self.height) ** 2))

if __name__ == "__main__":
    WIDTH, HEIGHT, FACTOR = 400, 400, 1
    shapes = [Shape(WIDTH, HEIGHT, FACTOR) for _ in range(0, 10)]
    for shape in shapes: print("{}".format(shape))

Color: [25, 226, 249, 87]; Radius: 47; Center: [377, 203]
Color: [220, 4, 120, 222]; Radius: 6; Center: [44, 388]
Color: [74, 234, 130, 80]; Radius: 16; Center: [234, 277]
Color: [234, 222, 174, 154]; Radius: 31; Center: [360, 377]
Color: [186, 129, 202, 66]; Radius: 13; Center: [362, 155]
Color: [33, 207, 11, 143]; Radius: 8; Center: [22, 256]
Color: [31, 112, 236, 56]; Radius: 7; Center: [237, 190]
Color: [137, 213, 155, 230]; Radius: 2; Center: [212, 265]
Color: [48, 169, 78, 18]; Radius: 53; Center: [199, 347]
Color: [75, 36, 216, 222]; Radius: 53; Center: [36, 274]


In [10]:
class Painting:
    '''
    '''
    def __init__(self, width: int, height: int, size: int) -> None:
        self.width = width
        self.height = height
        self.size = size
        self.canvas_genotype = Canvas()
        self.shape_genotype = [Shape(width = self.width,
                                     height = self.height,
                                     factor = 2) for _ in range(0, self.size)]

        self.overall_fitness = 0
        self.r_fitness = 0
        self.g_fitness = 0
        self.b_fitness = 0
        self.create_painting()
    
    def create_painting(self) -> None:
        self.sort_genotype(mode = '5')
        self.phenotype = Image.new(mode = 'RGB',
                                   size = (self.width, self.height),
                                   color = tuple(self.canvas_genotype.color))
        self.draw = ImageDraw.Draw(self.phenotype, 'RGBA')
        for shape in self.shape_genotype: shape.draw_circle(self.draw)
    
    def show_painting(self) -> None:
        self.phenotype.show()
    
    def save_painting(self, painting_id: int) -> None:
        '''
        - saves the populated image as a png file
        '''
        self.phenotype.save('../lib/benchmark2_test/test_painting{}.png'.format(str(painting_id)))
    
    def crossover(self, parent_canvas_gen: object, parent_shape_gen: object, mutation_rate: float) -> object:
        child = Painting(width = self.width, height = self.height, size = self.size)
        # CANVAS CROSSOVER
        child.canvas_genotype.color[0] = int(0.5 * (self.canvas_genotype.color[0] + parent_canvas_gen.color[0]))
        child.canvas_genotype.color[1] = int(0.5 * (self.canvas_genotype.color[1] + parent_canvas_gen.color[1]))
        child.canvas_genotype.color[2] = int(0.5 * (self.canvas_genotype.color[2] + parent_canvas_gen.color[2]))
        # SHAPE CROSSOVER
        if random.choice([0, 1]) == 0:
            '''
            - the first half of parent 1 is combined with the second half of parent 2
            '''
            parent1_traits = self.shape_genotype[: int(len(self.shape_genotype) / 2)]
            parent2_traits = parent_shape_gen[int(len(parent_shape_gen) / 2) :]
            child.shape_genotype = parent1_traits + parent2_traits
        else:
            '''
            - the second half of parent 1 is combined with the first half of parent 2
            '''
            parent1_traits = self.shape_genotype[int(len(self.shape_genotype) / 2) :]
            parent2_traits = parent_shape_gen[: int(len(parent_shape_gen) / 2)]
            child.shape_genotype = parent1_traits + parent2_traits
        child.canvas_mutate(mutation_rate = mutation_rate)
        child.shape_mutate(mutation_rate = mutation_rate, mode = '5')
        return child
    
    def canvas_mutate(self, mutation_rate: float) -> None:
        for index in range(0, len(self.canvas_genotype.color)):
            if random.randint(1, int(1 / mutation_rate)) == 1:
                self.canvas_genotype.color[index] = self.canvas_genotype.color[index] + random.randint(-30, 30)
    
    def shape_mutate(self, mutation_rate: float, mode: str) -> None:
        '''
        | specs:
        | - design in modes to compare methods
        | - "for" = for shape in shapes...
        | - "if" = if a shape is to be mutated or not
        | - "choose" = choose one of the mutation types
        | - extensive documentation!!!
        '''
        mutation_types = ['shape', 'gene']
        weights = [0.99, 0.01]
        mutation = random.choices(mutation_types, weights = weights, k = 1)[0]
        if mutation == 'shape':
            if mode == '1':
                ''' for -> if -> choose'''
                for shape in self.shape_genotype:
                    if random.randint(1, int(1 / mutation_rate)) == 1:
                        shape_mut_types = ['radius', 'shift', 'R', 'G', 'B', 'A']
                        shape_weights = [0.10, 0.10, 0.20, 0.20, 0.20, 0.20]
                        shape_mutation = random.choices(shape_mut_types, 
                                                        weights = shape_weights, 
                                                        k = 1)[0]
                        self.choose_mutation(obj = shape, mut_type = shape_mutation)
            elif mode == '2':
                ''' for -> choose -> if'''
                for shape in self.shape_genotype:
                    shape_mut_types = ['radius', 'shift', 'R', 'G', 'B', 'A']
                    shape_weights = [0.10, 0.10, 0.20, 0.20, 0.20, 0.20]
                    shape_mutation = random.choices(shape_mut_types, 
                                                    weights = shape_weights, 
                                                    k = 1)[0]
                    if random.randint(1, int(1 / mutation_rate)) == 1:
                        shape.choose_mutation(obj = shape, mut_type = shape_mutation)
            elif mode == '3':
                ''' choose -> for -> if'''
                shape_mut_types = ['radius', 'shift', 'R', 'G', 'B', 'A']
                shape_weights = [0.10, 0.10, 0.20, 0.20, 0.20, 0.20]
                shape_mutation = random.choices(shape_mut_types, 
                                                weights = shape_weights, 
                                                k = 1)[0]
                for shape in self.shape_genotype:
                    if random.randint(1, int(1 / mutation_rate)) == 1:
                        shape.choose_mutation(obj = shape, mut_type = shape_mutation)
            elif mode == '4':
                ''' choose -> if -> for'''
                shape_mut_types = ['radius', 'shift', 'R', 'G', 'B', 'A']
                shape_weights = [0.10, 0.10, 0.20, 0.20, 0.20, 0.20]
                shape_mutation = random.choices(shape_mut_types, 
                                                weights = shape_weights, 
                                                k = 1)[0]
                if random.randint(1, int(1 / mutation_rate)) == 1:
                    for shape in self.shape_genotype:
                        shape.choose_mutation(obj = shape, mut_type = shape_mutation)
            elif mode == '5':
                '''if -> choose -> for'''
                if random.randint(1, int(1 / mutation_rate)) == 1:
                    shape_mut_types = ['radius', 'shift', 'R', 'G', 'B', 'A']
                    shape_weights = [0.10, 0.10, 0.20, 0.20, 0.20, 0.20]
                    shape_mutation = random.choices(shape_mut_types, 
                                                    weights = shape_weights, 
                                                    k = 1)[0]
                    for shape in self.shape_genotype:
                        self.choose_mutation(obj = shape, mut_type = shape_mutation)
            elif mode == '6':
                ''' if -> for -> choose'''
                if random.randint(1, int(1 / mutation_rate)) == 1:
                    for shape in self.shape_genotype:
                        shape_mut_types = ['radius', 'shift', 'R', 'G', 'B', 'A']
                        shape_weights = [0.10, 0.10, 0.20, 0.20, 0.20, 0.20]
                        shape_mutation = random.choices(shape_mut_types, 
                                                        weights = shape_weights, 
                                                        k = 1)[0]
                        self.choose_mutation(obj = shape, mut_type = shape_mutation)
            else:
                '''choose a default means of mutation'''
                pass
        else:
            gene_mutation_types = ['add', 'subtract']
            gene_weights = [0.85, 0.15]
            gene_mutation = random.choices(gene_mutation_types, weights = gene_weights, k = 1)[0]
            if gene_mutation == 'add':
                self.shape_genotype.append(self.create_new_gene(factor = 0.5))
            else:
                if self.shape_genotype: self.shape_genotype.pop(0)
    
    def choose_mutation(self, obj: object, mut_type: str) -> None:
        if mut_type == 'radius':
            self.mutate_radius(shape = obj)
        elif mut_type == 'shift':
            self.mutate_coordinates(shape = obj)
        elif mut_type == 'R':
            self.mutate_color_R(shape = obj)
        elif mut_type == 'G':
            self.mutate_color_G(shape = obj)
        elif mut_type == 'B':
            self.mutate_color_B(shape = obj)
        elif mut_type == 'A':
            self.mutate_color_A(shape = obj)
        else:
            pass

    def mutate_coordinates(self, shape: object) -> None:
        x_shift = random.randint(-5, 5)
        y_shift = random.randint(-5, 5)
        shape.center = [shape.center[0] + x_shift, shape.center[1] + y_shift]
        shape.center[0] = min(max(shape.center[0], 0), shape.width - 1)
        shape.center[1] = min(max(shape.center[1], 0), shape.height - 1)
    
    def mutate_color_R(self, shape: object) -> None:
        R_shift = random.randint(-10, 10)
        shape.color[0] = shape.color[0] + R_shift
        shape.color[0] = min(max(shape.color[0], 0), 255)
        
    def mutate_color_G(self, shape: object) -> None:
        G_shift = random.randint(-10, 10)
        shape.color[1] = shape.color[1] + G_shift
        shape.color[1] = min(max(shape.color[1], 0), 255)
        
    def mutate_color_B(self, shape: object) -> None:
        B_shift = random.randint(-10, 10)
        shape.color[2] = shape.color[2] + B_shift
        shape.color[2] = min(max(shape.color[2], 0), 255)
        
    def mutate_color_A(self, shape: object) -> None:
        A_shift = random.randint(-10, 10)
        shape.color[3] = shape.color[3] + A_shift
        shape.color[3] = min(max(shape.color[3], 0), 255)
    
    def mutate_radius(self, shape: object) -> None:
        shift = random.randint(-1, 1)
        shape.radius = shape.radius + shift
    
    def sort_genotype(self, mode: int) -> None:
        '''
        | specs:
        | - design in modes to compare methods
        | - extensive documentation!!!
        | - mode #1: larger, more opaque shapes are drawn first
        | - mode #2: smaller, less opaque shapes are drawn first
        | - mode #3: smaller, more opaque shapes are drawn first
        | - mode #4: larger, less opaque shapes are drawn first
        |
        | - mode #5: larger shapes are drawn first
        | - mode #6: smaller shapes are drawn first
        | - mode #7: more opaque shapes are drawn first
        | - mode #8: less opaque shapes are drawn first
        '''
        if mode == '1':
            sort_func = lambda shape: (0.5 * shape.radius) + (0.5 * shape.color[-1])
            self.shape_genotype.sort(reverse = True, key = sort_func)
        elif mode == '2':
            func = lambda shape: (0.5 * shape.radius) + (0.5 * shape.color[-1])
            self.shape_genotype.sort(reverse = False, key = func)
        elif mode == '3':
            max_rad = max([shape.radius for shape in self.shape_genotype])
            sort_func = lambda shape: (0.5 * (max_rad - shape.radius)) + (0.5 * shape.color[-1])
            self.shape_genotype.sort(reverse = True, key = sort_func)
        elif mode == '4':
            max_opac = max([shape.color[-1] for shape in self.shape_genotype])
            sort_func = lambda shape: (0.5 * (max_opac - shape.color[-1])) + (0.5 * shape.radius)
            self.shape_genotype.sort(reverse = True, key = sort_func)
        elif mode == '5':
            self.shape_genotype.sort(reverse = True, key = lambda shape: shape.radius)
        elif mode == '6':
            self.shape_genotype.sort(reverse = False, key = lambda shape: shape.radius)
        elif mode == '7':
            self.shape_genotype.sort(reverse = True, key = lambda shape: shape.color[-1])
        elif mode == '8':
            self.shape_genotype.sort(reverse = False, key = lambda shape: shape.color[-1])
        else:
            '''opt for a random placement of shapes onto the canvas'''
            pass
    
    def create_new_gene(self, factor):
        '''
        - probablistically, an entire new gene will be generated
        '''
        return Shape(width = self.width, height = self.height, factor = factor)


if __name__ == "__main__":
    painting = Painting(width = 500, height = 500, size = 500)
    painting.show_painting()
    print('{}'.format(painting.canvas_genotype))
    for shape in painting.shape_genotype: print('{}'.format(shape))

Color: [219, 174, 154]
Color: [7, 3, 253, 146]; Radius: 327; Center: [166, 97]
Color: [121, 38, 230, 125]; Radius: 292; Center: [11, 421]
Color: [82, 93, 14, 180]; Radius: 265; Center: [42, 331]
Color: [217, 202, 213, 40]; Radius: 245; Center: [491, 330]
Color: [49, 144, 59, 234]; Radius: 236; Center: [232, 293]
Color: [105, 125, 251, 187]; Radius: 228; Center: [381, 54]
Color: [164, 155, 208, 64]; Radius: 220; Center: [140, 374]
Color: [239, 16, 222, 29]; Radius: 211; Center: [387, 25]
Color: [187, 125, 41, 29]; Radius: 205; Center: [94, 454]
Color: [56, 192, 8, 141]; Radius: 203; Center: [223, 117]
Color: [242, 222, 254, 221]; Radius: 202; Center: [204, 308]
Color: [185, 39, 225, 187]; Radius: 202; Center: [340, 312]
Color: [168, 215, 243, 128]; Radius: 190; Center: [128, 70]
Color: [118, 166, 15, 16]; Radius: 187; Center: [347, 85]
Color: [84, 18, 65, 253]; Radius: 181; Center: [117, 379]
Color: [14, 34, 205, 81]; Radius: 178; Center: [413, 194]
Color: [99, 141, 174, 164]; Radius: 1

In [11]:
class Figure:
    '''
    '''
    def __init__(self, path: str) -> None:
        self.path = path
        self.image = Image.open(self.path)
        self.image = self.convert_image()
        self.width, self.height = self.image.size
        self.data = self.get_color_data(self.image)
        self.paint_data = []
    
    @staticmethod
    def get_color_data(image: object) -> list:
        return np.array(Image.Image.getdata(image))

    def calculate_overall_fitness(self, input_painting: object) -> float:
        '''
        - for each pixel, determine the mean error of the color map
        '''
        input_painting = input_painting.phenotype.convert('RGB')
        self.paint_data = self.get_color_data(input_painting)
        diff1_percent = imgcompare.image_diff_percent(self.image, input_painting)
        return 100 - diff1_percent
    
    def calculate_r_fitness(self) -> float:
        return 255 - np.mean(np.abs(self.paint_data[:, 0] - self.data[:, 0]))
    
    def calculate_g_fitness(self) -> float:
        return 255 - np.mean(np.abs(self.paint_data[:, 1] - self.data[:, 1]))
    
    def calculate_b_fitness(self) -> float:
        return 255 - np.mean(np.abs(self.paint_data[:, 2] - self.data[:, 2]))

    def convert_image(self) -> object:
        return self.image.convert('RGB')
        

if __name__ == "__main__":
    target = Figure('../lib/test_target1.png')
    test_p = Painting(target.width, target.height, 3)
    print('{}'.format(target.data.shape))
    print('{}'.format(target.data[:30]))
    print('Overall Fitness: {}'.format(target.calculate_overall_fitness(test_p)))
    print('R Fitness: {}'.format(target.calculate_r_fitness()))
    print('G Fitness: {}'.format(target.calculate_g_fitness()))
    print('B Fitness: {}'.format(target.calculate_b_fitness()))

(45000, 3)
[[ 4  4 17]
 [ 3  4 16]
 [ 3  4 15]
 [ 3  4 14]
 [ 3  4 14]
 [ 1  4 14]
 [ 2  4 14]
 [ 3  4 14]
 [ 3  4 15]
 [ 3  4 16]
 [ 4  4 17]
 [ 4  4 18]
 [ 5  5 20]
 [ 5  5 22]
 [ 7  5 24]
 [ 8  5 26]
 [ 9  5 29]
 [11  5 31]
 [12  5 33]
 [12  6 35]
 [13  6 37]
 [15  6 39]
 [16  6 42]
 [16  6 43]
 [17  6 45]
 [18  7 47]
 [18  7 49]
 [19  7 51]
 [20  7 53]
 [20  7 54]]
Overall Fitness: 67.47547712418302
R Fitness: 177.36913333333334
G Fitness: 164.05793333333332
B Fitness: 199.459


In [12]:
class Population:
    '''
    '''
    def __init__(self, max_population: int, mutation_rate: float, target_path: str, size: int) -> None:
        self.max_population = max_population
        self.mutation_rate = mutation_rate
        self.target_path = target_path
        self.size = size
        self.target = Figure(self.target_path)
        self.population = [Painting(width = self.target.width,
                                    height = self.target.height,
                                    size = self.size) for _ in range(0, self.max_population)]
        self.children = []
    
    def choose_parent(self) -> object:
        weights = [painting.fitness for painting in self.population]
        return random.choices(self.population, weights = weights, k = 1)[0]
    
    def evaluate_population(self) -> None:
        for painting in self.population:
            painting.create_painting()
            painting.overall_fitness = self.target.calculate_overall_fitness(painting)
            painting.r_fitness = self.target.calculate_r_fitness() + painting.overall_fitness
            painting.g_fitness = self.target.calculate_g_fitness() + painting.overall_fitness
            painting.b_fitness = self.target.calculate_b_fitness() + painting.overall_fitness
    
    def generate_next_generation(self) -> None:
        '''
        - sexual reproduction
        - organisms in the population must be unique in order to mate
        '''
        # separate pop into 3 arrays
        # perform 3 crossovers (r, g, b)
        # add up 3 children arrays into one children array
        self.children = []
        r_children = []
        g_children = []
        b_children = []
        self.scale_fitness()

        self.sort_fitness(mode = 'R')
        for _ in range(0, len(self.population), 6):
            parent1 = self.choose_parent()
            parent2 = self.choose_parent()
            while parent1 == parent2: parent2 = self.choose_parent()
            child = parent1.crossover(parent_canvas_gen = parent2.canvas_genotype, 
                                      parent_shape_gen = parent2.shape_genotype, 
                                      mutation_rate = self.mutation_rate)
            r_children.append(child)
        
        self.sort_fitness(mode = 'G')
        for _ in range(0, len(self.population), 6):
            parent1 = self.choose_parent()
            parent2 = self.choose_parent()
            while parent1 == parent2: parent2 = self.choose_parent()
            child = parent1.crossover(parent_canvas_gen = parent2.canvas_genotype,
                                      parent_shape_gen = parent2.shape_genotype, 
                                      mutation_rate = self.mutation_rate)
            g_children.append(child)
        
        self.sort_fitness(mode = 'B')
        for _ in range(0, len(self.population), 6):
            parent1 = self.choose_parent()
            parent2 = self.choose_parent()
            while parent1 == parent2: parent2 = self.choose_parent()
            child = parent1.crossover(parent_canvas_gen = parent2.canvas_genotype,
                                      parent_shape_gen = parent2.shape_genotype, 
                                      mutation_rate = self.mutation_rate)
            b_children.append(child)
        
        for index in r_children: self.children.append(index)
        for index in g_children: self.children.append(index)
        for index in b_children: self.children.append(index)

    def adjust_population(self) -> None:
        if random.choice([0, 1]) == 0:
            self.population = self.population[: int(len(self.population) / 2)] + self.children
        else:
            self.population = self.population[int(len(self.population) / 2) :] + self.children
    
    def sort_fitness(self, mode: str) -> None:
        if mode == 'R':
            self.population.sort(reverse = True, key = lambda painting: painting.r_fitness)
        elif mode == 'G':
            self.population.sort(reverse = True, key = lambda painting: painting.g_fitness)
        elif mode == 'B':
            self.population.sort(reverse = True, key = lambda painting: painting.b_fitness)
        else: # SORT OVERALL FITNESS
            self.population.sort(reverse = True, key = lambda painting: painting.overall_fitness)
    
    def scale_fitness(self) -> None:
        mode = 'x'
        if mode == 'R': pass
        elif mode == 'G': pass
        elif mode == 'B': pass
        else:
            self.sort_fitness(mode = 'X')
            for i in range(0, len(self.population)):
                self.population[i].fitness = int(10 * (self.max_population / ((i + 1) ** 0.5)))

if __name__ == "__main__":
    pop = Population(max_population = 100, 
                     mutation_rate = 0.05, 
                     target_path = '../lib/test_target0.png', 
                     size = 3)

    pop.evaluate_population()
    pop.sort_fitness(mode = 'R')
    print('Best R fitness: {}'.format(pop.population[0].r_fitness))

    pop.sort_fitness(mode = 'G')
    print('Best G fitness: {}'.format(pop.population[0].g_fitness))

    pop.sort_fitness(mode = 'B')
    print('Best B fitness: {}'.format(pop.population[0].b_fitness))

    pop.sort_fitness(mode = 'X')
    print('Best overall fitness: {}'.format(pop.population[0].overall_fitness))

Best R fitness: 240.0445350762527
Best G fitness: 241.71226608569356
Best B fitness: 228.33608830791576
Best overall fitness: 66.96210312273058


In [13]:
class Evolution:
    '''
    '''
    def __init__(self) -> None:
        self.pop = Population(max_population = 60,
                              mutation_rate = 0.20,
                              target_path = '../lib/test_target0.png',
                              size = 5)
        self.generation = 1
        self.best_fitness = 0
    
    def evolve(self) -> None:
        count = 0
        # implement a canvas evolution
        # implement a shape evolution
        while count < 1000:
            self.pop.sort_fitness(mode = 'X')
            self.pop.population[0].save_painting(self.generation)
            self.pop.evaluate_population()
            self.save_best_painting()
            self.display_stats()
            self.pop.generate_next_generation()
            self.pop.adjust_population()
            self.generation += 1
            count += 1
    
    def save_best_painting(self) -> None:
        if self.pop.population[0].overall_fitness > self.best_fitness:
            self.pop.population[0].save_painting('_best')
            self.best_fitness = self.pop.population[0].overall_fitness
    
    def display_stats(self) -> None:
        print("--------------- STATS ---------------")
        print("Generation: {}".format(self.generation))
        print("Population Size: {}".format(len(self.pop.population)))
        print("Mean Fitness: {}".format(np.mean([i.overall_fitness for i in self.pop.population])))
        print("Std Fitness: {}".format(np.std([i.overall_fitness for i in self.pop.population])))
        print("Max Fitness: {}".format(max([i.overall_fitness for i in self.pop.population])))
        print("Min Fitness: {}".format(min([i.overall_fitness for i in self.pop.population])))
        print("-------------------------------------")

if __name__ == "__main__":
    evo = Evolution()
    evo.evolve()

--------------- STATS ---------------
Generation: 1
Population Size: 60
Mean Fitness: 52.22962149600581
Std Fitness: 5.245943642486015
Max Fitness: 62.68047833454369
Min Fitness: 37.69953231663036
-------------------------------------
--------------- STATS ---------------
Generation: 2
Population Size: 60
Mean Fitness: 54.35738626644073
Std Fitness: 4.67425939338579
Max Fitness: 62.67438973614137
Min Fitness: 41.94530718954248
-------------------------------------
--------------- STATS ---------------
Generation: 3
Population Size: 60
Mean Fitness: 56.717323650447845
Std Fitness: 4.714535304794299
Max Fitness: 65.18948438634713
Min Fitness: 41.94530718954248
-------------------------------------
--------------- STATS ---------------
Generation: 4
Population Size: 60
Mean Fitness: 57.72465811345114
Std Fitness: 4.126982284521022
Max Fitness: 65.14718954248366
Min Fitness: 42.692903413217145
-------------------------------------
--------------- STATS ---------------
Generation: 5
Populat

--------------- STATS ---------------
Generation: 36
Population Size: 60
Mean Fitness: 68.1017379488421
Std Fitness: 1.1784410286367102
Max Fitness: 70.71700217864924
Min Fitness: 65.21282401355604
-------------------------------------
--------------- STATS ---------------
Generation: 37
Population Size: 60
Mean Fitness: 68.70864494472686
Std Fitness: 0.9252259562947044
Max Fitness: 70.97974921326555
Min Fitness: 65.65001791333817
-------------------------------------
--------------- STATS ---------------
Generation: 38
Population Size: 60
Mean Fitness: 68.4048533204228
Std Fitness: 0.815223249872183
Max Fitness: 70.35144613894941
Min Fitness: 66.47847785039943
-------------------------------------
--------------- STATS ---------------
Generation: 39
Population Size: 60
Mean Fitness: 68.38917014443638
Std Fitness: 0.9494260896070049
Max Fitness: 71.25214814814815
Min Fitness: 66.1119147906076
-------------------------------------
--------------- STATS ---------------
Generation: 40
Pop

--------------- STATS ---------------
Generation: 71
Population Size: 60
Mean Fitness: 67.1906427176632
Std Fitness: 0.38519362782616334
Max Fitness: 67.95401016702978
Min Fitness: 66.02748002904866
-------------------------------------
--------------- STATS ---------------
Generation: 72
Population Size: 60
Mean Fitness: 67.11562759622366
Std Fitness: 0.3749907826440629
Max Fitness: 67.8723330912612
Min Fitness: 65.96687291212783
-------------------------------------
--------------- STATS ---------------
Generation: 73
Population Size: 60
Mean Fitness: 67.36569715161785
Std Fitness: 0.3586277335044249
Max Fitness: 67.94642653110627
Min Fitness: 66.11009440813362
-------------------------------------
--------------- STATS ---------------
Generation: 74
Population Size: 60
Mean Fitness: 67.28949994351652
Std Fitness: 0.3400685409906821
Max Fitness: 67.92280029048656
Min Fitness: 66.07349310094408
-------------------------------------
--------------- STATS ---------------
Generation: 75


--------------- STATS ---------------
Generation: 106
Population Size: 60
Mean Fitness: 70.7218147018478
Std Fitness: 0.33161994368086395
Max Fitness: 71.2553163882837
Min Fitness: 69.67860953764222
-------------------------------------
--------------- STATS ---------------
Generation: 107
Population Size: 60
Mean Fitness: 70.13037708383764
Std Fitness: 0.30440719617858925
Max Fitness: 70.65899007504237
Min Fitness: 69.17077705156137
-------------------------------------
--------------- STATS ---------------
Generation: 108
Population Size: 60
Mean Fitness: 70.2408976680384
Std Fitness: 0.3986435516572773
Max Fitness: 70.82816170418785
Min Fitness: 68.67595061728395
-------------------------------------
--------------- STATS ---------------
Generation: 109
Population Size: 60
Mean Fitness: 70.32747544581618
Std Fitness: 0.335742612556706
Max Fitness: 70.74044250786734
Min Fitness: 68.62879883805374
-------------------------------------
--------------- STATS ---------------
Generation: 

--------------- STATS ---------------
Generation: 141
Population Size: 60
Mean Fitness: 69.27266966836117
Std Fitness: 0.08385294589948335
Max Fitness: 69.32700459937061
Min Fitness: 68.89885064149117
-------------------------------------
--------------- STATS ---------------
Generation: 142
Population Size: 60
Mean Fitness: 72.67895037521183
Std Fitness: 0.05957946137966502
Max Fitness: 72.6955642701525
Min Fitness: 72.36338707334785
-------------------------------------
--------------- STATS ---------------
Generation: 143
Population Size: 60
Mean Fitness: 72.79285364318567
Std Fitness: 0.058862575571892964
Max Fitness: 72.81129411764707
Min Fitness: 72.4876456063907
-------------------------------------
--------------- STATS ---------------
Generation: 144
Population Size: 60
Mean Fitness: 72.50879173727104
Std Fitness: 0.0587287654072848
Max Fitness: 72.5439690147664
Min Fitness: 72.21361994674413
-------------------------------------
--------------- STATS ---------------
Generatio

...

--------------- STATS ---------------
Generation: 841
Population Size: 60
Mean Fitness: 66.80114955216655
Std Fitness: 0.0018453828428736984
Max Fitness: 66.80474461389494
Min Fitness: 66.7923815056887
-------------------------------------
--------------- STATS ---------------
Generation: 842
Population Size: 60
Mean Fitness: 66.93496560961835
Std Fitness: 0.012536563394103088
Max Fitness: 66.94348874364562
Min Fitness: 66.84075720164608
-------------------------------------
--------------- STATS ---------------
Generation: 843
Population Size: 60
Mean Fitness: 67.36068748487048
Std Fitness: 0.0027199845494167087
Max Fitness: 67.36708012587751
Min Fitness: 67.35794722827401
-------------------------------------
--------------- STATS ---------------
Generation: 844
Population Size: 60
Mean Fitness: 67.49867738239328
Std Fitness: 0.0016244343582086482
Max Fitness: 67.50282643427741
Min Fitness: 67.49349987896393
-------------------------------------
--------------- STATS ---------------


--------------- STATS ---------------
Generation: 876
Population Size: 60
Mean Fitness: 66.19277049947551
Std Fitness: 0.18144373605381733
Max Fitness: 66.8232273057371
Min Fitness: 65.2001433067054
-------------------------------------
--------------- STATS ---------------
Generation: 877
Population Size: 60
Mean Fitness: 66.76788522553053
Std Fitness: 0.17052325002519836
Max Fitness: 67.17522730573711
Min Fitness: 65.56361558944565
-------------------------------------
--------------- STATS ---------------
Generation: 878
Population Size: 60
Mean Fitness: 66.53567749536028
Std Fitness: 0.20152171734753987
Max Fitness: 66.90917259743404
Min Fitness: 65.1999264100702
-------------------------------------
--------------- STATS ---------------
Generation: 879
Population Size: 60
Mean Fitness: 66.77333514080529
Std Fitness: 0.10234525645818124
Max Fitness: 67.11726942628903
Min Fitness: 66.59662454611474
-------------------------------------
--------------- STATS ---------------
Generatio

--------------- STATS ---------------
Generation: 911
Population Size: 60
Mean Fitness: 65.82593115468408
Std Fitness: 0.25308842377436197
Max Fitness: 66.21714064391189
Min Fitness: 65.46859937061244
-------------------------------------
--------------- STATS ---------------
Generation: 912
Population Size: 60
Mean Fitness: 66.36955145646736
Std Fitness: 0.2856361741383189
Max Fitness: 66.88589494069232
Min Fitness: 66.06725732268217
-------------------------------------
--------------- STATS ---------------
Generation: 913
Population Size: 60
Mean Fitness: 65.81129502138303
Std Fitness: 0.32639475918694866
Max Fitness: 66.20312757201646
Min Fitness: 65.39457564754298
-------------------------------------
--------------- STATS ---------------
Generation: 914
Population Size: 60
Mean Fitness: 65.50785921084484
Std Fitness: 0.3030058498052469
Max Fitness: 65.7125306221254
Min Fitness: 64.83750762527234
-------------------------------------
--------------- STATS ---------------
Generatio

--------------- STATS ---------------
Generation: 946
Population Size: 60
Mean Fitness: 66.34343426127653
Std Fitness: 0.2569839100076206
Max Fitness: 67.06312273057371
Min Fitness: 65.4639748244977
-------------------------------------
--------------- STATS ---------------
Generation: 947
Population Size: 60
Mean Fitness: 65.60641923666587
Std Fitness: 0.2708033059496096
Max Fitness: 66.39401984991528
Min Fitness: 64.72491503267975
-------------------------------------
--------------- STATS ---------------
Generation: 948
Population Size: 60
Mean Fitness: 65.89381941418543
Std Fitness: 0.28864688990648646
Max Fitness: 66.64250593076737
Min Fitness: 65.0250612442508
-------------------------------------
--------------- STATS ---------------
Generation: 949
Population Size: 60
Mean Fitness: 66.83264074880981
Std Fitness: 0.26279599190578584
Max Fitness: 67.41670297748729
Min Fitness: 65.84612345679012
-------------------------------------
--------------- STATS ---------------
Generation

--------------- STATS ---------------
Generation: 981
Population Size: 60
Mean Fitness: 61.86883395465182
Std Fitness: 0.11374154092007654
Max Fitness: 62.058542725732266
Min Fitness: 61.58515032679738
-------------------------------------
--------------- STATS ---------------
Generation: 982
Population Size: 60
Mean Fitness: 62.948555345759715
Std Fitness: 0.12227359948879082
Max Fitness: 63.120817235536194
Min Fitness: 62.67625659646575
-------------------------------------
--------------- STATS ---------------
Generation: 983
Population Size: 60
Mean Fitness: 63.00844722020495
Std Fitness: 0.1186581416805535
Max Fitness: 63.09368191721133
Min Fitness: 62.517255870249336
-------------------------------------
--------------- STATS ---------------
Generation: 984
Population Size: 60
Mean Fitness: 62.35276339869281
Std Fitness: 0.12935053545929717
Max Fitness: 62.471630113773905
Min Fitness: 61.90978261922053
-------------------------------------
--------------- STATS ---------------
Ge